In [49]:
import yfinance as yf  #Start up cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
#Setting up the strategy
START = "2022-01-01"

SECTOR_ETF = "JETS"
TICKERS = ["AAL", "DAL", "UAL", "LUV"]

CAPITAL0 = 10_000 #Starting capital

DTE = 10     #Iron condor information
WING_WIDTH = 2.5                #Dollar difference between short and long option
RISK_REWARD = 3        #Assumed payoff for each condor 3:1
CREDIT = WING_WIDTH / (1 + RISK_REWARD) #Assumed credit collected

# Entry filters
VOL_RATIO_ENTRY = 1.2            # stock RV20/RV252 must be >= this
SECTOR_RATIO_MAX = 1.15             # JETS RV20/RV252 must be <= this
ADX_THRESH = 25
MIN_RV20 = 0.20                     # optional absolute vol guardrail

In [51]:
# Strike rounding 0.5 strikes when sp < 50
def choose_strike_increment(price):
    return 0.5 if price < 50 else 1.0

#Rounds the strike based on the increment
def round_to_increment(x, inc):
    return round(x / inc) * inc


In [47]:
def realized_vol(close, window): #This function is calculating the annualized volatility
    r = np.log(close).diff() #It takes the logarithmic return
    return r.rolling(window).std() * np.sqrt(252) #This takes the standard deviation of returns over window period and annualizes it

In [48]:
def adx(high, low, close, n=14):
    # Wilder ADX (price-only)
    up_move = high.diff() #Did we go higher than yesterday?
    down_move = -low.diff() #Did we go lower than yesterday?

    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0) #If we moved up more bulls get point
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0) #If we moved down more bears get points

    tr1 = (high - low) #Todays high - low
    tr2 = (high - close.shift()).abs() #Todays high - yesterdays close
    tr3 = (low - close.shift()).abs() #Todays low - yesterdays close
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1) #Which one of these three is the greatest?

    atr = tr.ewm(alpha=1/n, adjust=False).mean() #it now converts the true range we just determined into ATR using EMA using 1/14 smoothing factor
    plus_di = 100 * pd.Series(plus_dm, index=high.index).ewm(alpha=1/n, adjust=False).mean() / atr #Now see how strong the move is by dividing by smoothing out and dividng by ATR
    minus_di = 100 * pd.Series(minus_dm, index=high.index).ewm(alpha=1/n, adjust=False).mean() / atr

    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di) #Calculates margin of victory between the bullish or bearish moves.
    return dx.ewm(alpha=1/n, adjust=False).mean() #Smooths it out and returns it

In [38]:
def condor_payoff_at_expiry(S_exp, Ksp, Klp, Ksc, Klc, credit): #Function to calculate how much profit we have at expiry

    w_put = Ksp - Klp #wing width redundant
    w_call = Klc - Ksc #wing width redundant
    w = max(w_put, w_call)  #Picks the bigger one to define max loss redundant

    # Max loss
    if S_exp <= Klp or S_exp >= Klc: #If the stock is below long put or above long call we hit max loss
        return -(w - credit)

    # Max profit #If the stock is in between the shorts we get max profit
    if Ksp <= S_exp <= Ksc:
        return credit

    # Calculating payoff if its in between the short and long
    if Klp < S_exp < Ksp:
        # from -(w-credit) up to +credit
        return (-(w - credit)) + (S_exp - Klp) * ((credit + (w - credit)) / (Ksp - Klp))

    if Ksc < S_exp < Klc:
        # from +credit down to -(w-credit)
        return credit + (S_exp - Ksc) * ((-(w - credit) - credit) / (Klc - Ksc))

    return 0.0


In [39]:
#Downloading yahoo finance data
all_symbols = TICKERS + [SECTOR_ETF]
raw = yf.download(all_symbols, start=START, auto_adjust=True, group_by="ticker")

dates = raw.index

# Build a data frame containing close, high low information for each date
data = {}
for sym in all_symbols:
    df = pd.DataFrame(index=dates)
    df["close"] = raw[(sym, "Close")]
    df["high"]  = raw[(sym, "High")]
    df["low"]   = raw[(sym, "Low")]
    data[sym] = df.dropna()


# Align to common dates across everything
common_index = data[SECTOR_ETF].index
for sym in TICKERS:
    common_index = common_index.intersection(data[sym].index)
for sym in all_symbols:
    data[sym] = data[sym].reindex(common_index).dropna()

[*********************100%***********************]  5 of 5 completed


In [40]:
#Setting up a data frame that houses all of our indicators and signals
jets = data[SECTOR_ETF].copy()
jets["rv20"] = realized_vol(jets["close"], 20)
jets["rv252"] = realized_vol(jets["close"], 252)
jets["jets_vol_ratio"] = jets["rv20"] / jets["rv252"]

ind = {}
for sym in TICKERS:
    df = data[sym].copy()
    df["rv20"] = realized_vol(df["close"], 20)
    df["rv252"] = realized_vol(df["close"], 252)
    df["vol_ratio"] = df["rv20"] / df["rv252"]
    df["adx14"] = adx(df["high"], df["low"], df["close"], 14)
    df["jets_vol_ratio"] = jets["jets_vol_ratio"]

    df["signal"] = (
        (df["vol_ratio"] >= VOL_RATIO_ENTRY) &
        (df["rv20"] >= MIN_RV20) &
        (df["jets_vol_ratio"] <= SECTOR_RATIO_MAX) &
        (df["adx14"] <= ADX_THRESH)
    )
    ind[sym] = df


In [41]:
#Full backtesting logic
capital = CAPITAL0
equity = pd.Series(CAPITAL0, index=common_index, dtype=float) #Tracking results

open_trades = []   # list of dicts
closed_trades = []

for i, date in enumerate(common_index):
    #First check if any trades have expiry date today
    if open_trades:
        still_open = []
        for t in open_trades:
            if t["exp_date"] == date:
                S_exp = float(ind[t["ticker"]].loc[date, "close"])
                payoff_per_share = condor_payoff_at_expiry(
                    S_exp, t["Ksp"], t["Klp"], t["Ksc"], t["Klc"], t["credit"]
                )
                pnl = payoff_per_share * 100  # Add closed trade to profit
                capital += pnl #Add back to capital

                closed_trades.append({
                    **t,
                    "S_exp": S_exp,
                    "pnl_$": float(pnl)
                }) #Add it to closed trades
            else:
                still_open.append(t)
        open_trades = still_open

    # Record the current equity amount
    equity.iloc[i] = capital if i == 0 else capital

    #Look at opening a trade
    exp_idx = i + DTE
    if exp_idx < len(common_index):
        exp_date = common_index[exp_idx] #Set the expiry date
        #Get activte tickers (dont wanna over lap trades)
        active_tickers = [t["ticker"] for t in open_trades]

        for sym in TICKERS:
            #Make sure ticker doesnt have an active trade
            if sym in active_tickers:
                continue
            #Check if all three signals are good
            if bool(ind[sym].loc[date, "signal"]):
                S0 = float(ind[sym].loc[date, "close"])
                rv20 = float(ind[sym].loc[date, "rv20"])

                # Estimating move(not sure what the logic behind formulas is)
                T = DTE / 252.0 #convert days into years
                STRIKE_AGGRESSION = 0.5   #
                sigma_T = rv20 * np.sqrt(T) * STRIKE_AGGRESSION #"Square Root of Time" Rule Calcs Expected % move volatility * time scaling factor Technically 0.5 sigma
                move_1sigma = S0 * sigma_T #convertes move to dollars

                # Short strikes
                Ksp = S0 - move_1sigma  #technically 0.5 sigma
                Ksc = S0 + move_1sigma

                inc = choose_strike_increment(S0)  #Using previous functions to round off strikes
                Ksp = float(round_to_increment(Ksp, inc))
                Ksc = float(round_to_increment(Ksc, inc))

                # Long strikes (using the wider wings you agreed on)
                Klp = float(Ksp - WING_WIDTH) #other strikes simple width
                Klc = float(Ksc + WING_WIDTH)

                open_trades.append({
                    "ticker": sym,
                    "entry_date": date,
                    "exp_date": exp_date,
                    "S_entry": S0,
                    "Klp": Klp, "Ksp": Ksp,
                    "Ksc": Ksc, "Klc": Klc,
                    "width": float(WING_WIDTH),
                    "credit": float(CREDIT),
                }) #Add trade to data

In [43]:

# ----------------------------
# Results: win rate + expected profit
# ----------------------------
trades_df = pd.DataFrame(closed_trades)
if len(trades_df) == 0:
    print("No trades were triggered. Try loosening thresholds or lowering MIN_RV20.")
else:
    total_pnl = trades_df["pnl_$"].sum()
    win_rate = (trades_df["pnl_$"] > 0).mean()
    expected_profit = trades_df["pnl_$"].mean()   # expectancy per trade (mean)
    median_profit = trades_df["pnl_$"].median()

    print(f"Start capital:      ${CAPITAL0:,.0f}")
    print(f"End capital:        ${(CAPITAL0 + total_pnl):,.2f}")
    print(f"Total PnL:          ${total_pnl:,.2f}")
    print(f"Number of trades:   {len(trades_df)}")
    print(f"Win rate:           {win_rate:.2%}")
    print(f"Expected profit/tr: ${expected_profit:.2f}")
    print(f"Median profit/tr:   ${median_profit:.2f}")

    display(trades_df.tail(10))


Start capital:      $10,000
End capital:        $10,299.00
Total PnL:          $299.00
Number of trades:   10
Win rate:           80.00%
Expected profit/tr: $29.90
Median profit/tr:   $62.50


,ticker,entry_date,exp_date,S_entry,Klp,Ksp,Ksc,Klc,width,credit,S_exp,pnl_$
0,LUV,2023-01-11,2023-01-26,33.638878,29.5,32.0,35.0,37.5,2.5,0.625,33.386375,62.500000
1,AAL,2024-02-12,2024-02-27,14.930000,11.5,14.0,15.5,18.0,2.5,0.625,15.540000,58.500004
2,AAL,2024-04-17,2024-05-01,13.890000,11.0,13.5,14.5,17.0,2.5,0.625,13.580000,62.500000
3,UAL,2024-04-17,2024-05-01,48.740002,43.0,45.5,52.0,54.5,2.5,0.625,50.669998,62.500000
4,UAL,2024-05-03,2024-05-17,51.650002,45.5,48.0,55.0,57.5,2.5,0.625,54.970001,62.500000
5,AAL,2024-05-29,2024-06-12,11.620000,8.5,11.0,12.5,15.0,2.5,0.625,11.500000,62.500000
6,LUV,2024-06-11,2024-06-26,26.970774,23.5,26.0,28.0,30.5,2.5,0.625,27.527611,62.500000
7,DAL,2025-01-10,2025-01-27,66.155579,61.5,64.0,69.0,71.5,2.5,0.625,66.965851,62.500000
8,DAL,2025-02-06,2025-02-21,67.114075,61.5,64.0,70.0,72.5,2.5,0.625,59.288055,-187.500000
9,AAL,2025-08-05,2025-08-19,11.640000,8.5,11.0,12.5,15.0,2.5,0.625,13.220000,-9.500027
